# Bayesian biodiversity: Geodata processing

In [189]:
import numpy as np
import pygeoprocessing as pgp
import geopandas as gpd
import pandas as pd
import rasterio
import shapely
from shapely import Point, Polygon, LineString, MultiLineString
import time
from rasterstats import zonal_stats
from pyproj import Transformer
from itertools import product
from osgeo import gdal, ogr
from typing import Tuple, Union, List
from datetime import timedelta
from osgeo import gdal, ogr, osr

In [2]:
# Load black for formatting
import jupyter_black

jupyter_black.load()

# Adjust display settings for pandas
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 100)

## Functions for performing geoprocessing steps

### Local projection, buffering and global reprojection

- TODO: Could be made more efficient by storing transformer objects in a dictionary, but requires a different handling with the apply functions.

In [155]:
def project_to_local_utm(
    geometry: Union[Point, LineString]
) -> Tuple[Union[Point, LineString], str, dict]:
    """
    Calculates the local UTM zone for a Point or LineString and transforms the
    gemoetry coordinates into local format.

    Args:
        geometry (Point or LineString): Coordinates of e.g. a sampling site
            or road segment.

    Returns:
        local_coords (Point or LineString): The input geometry transformed to
            local coordinates.
        epsg_code (str): The local EPSG code for this tranformation, used for
            reprojection in a later stage.
    """

    assert isinstance(
        geometry, (Point, LineString)
    ), "geometry should be a Point or LineString"

    try:
        # Get the coordinate point values (based on first point for Linestrings
        first_point = geometry.coords[0]
        long, lat = first_point[0], first_point[1]

        # Determine the UTM zone and hemisphere
        zone_number = int((long + 180) // 6) + 1
        epsg_code = f"EPSG:{32700 + zone_number if lat < 0 else 32600 + zone_number}"

        # Initialize Transformer object
        utm_transformer = Transformer.from_crs("EPSG:4326", epsg_code, always_xy=True)

        # Perform transformation with approach depending on type of geometry
        if isinstance(geometry, Point):
            # Perform transformation
            local_coords = Point(utm_transformer.transform(long, lat))
            return local_coords, epsg_code

        elif isinstance(geometry, LineString):
            xx, yy = geometry.coords.xy
            xx_utm, yy_utm = utm_transformer.transform(xx=xx, yy=yy)
            local_coords = LineString([xy for xy in zip(xx_utm, yy_utm)])
            return local_coords, epsg_code

    except ValueError as e:
        print(f"{geometry}: {e}")
        return None, None

In [119]:
def buffer_points_in_utm(
    points: gpd.GeoSeries, buffer_dist: int, polygon_type: str = "square"
) -> gpd.GeoSeries:
    """
    Creates a Polygon from Point coordinates, by buffering according to the
    desired radius.

    Args:
        points (GeoSeries): A GeoSeries with all the points that should be buffered
            into Polygons.
        buffer_dist (int): Buffer radius expressed in kilometers.
        polygon_type (str): The shape of the buffered Polygon. Can be any of
            ['square', 'round', 'flat'].

    Returns:
        utm_coords_buff (GeoSeries): The buffered points.
    """
    
    assert polygon_type in [
        "square",
        "round",
        "flat",
    ], "polygon_type must be one of ['square', 'round', 'flat']"

    # Buffer array of Points into the chosen size and type
    utm_coords_buff = shapely.buffer(points, buffer_dist, cap_style=polygon_type)

    return utm_coords_buff

In [120]:
def reproject_to_global(polygon: Polygon, epsg_code: str) -> Polygon:
    """
    Takes a Polygon defined by local UTM coordinates and reprojects it to global
    EPSG 4326 coordinates.

    Args:
        polygon (Polygon): The buffered site that should be reprojected.
        epsg_code (str): The local EPSG code for this tranformation.

    Returns:
        global_polygon (Polygon): The buffered site in global coordinates.
    """

    # Initialize Transformer object
    global_transformer = Transformer.from_crs(epsg_code, "EPSG:4326", always_xy=True)

    # Get the coordinates of the Polygon and perform transformation
    xx, yy = polygon.exterior.coords.xy
    xx, yy = global_transformer.transform(xx=xx, yy=yy)

    # Create a new Polygon from the reprojected coordinates
    global_coords = [xy for xy in zip(xx, yy)]
    global_polygon = shapely.Polygon(global_coords)

    return global_polygon

In [132]:
def run_site_buffering(
    gdf: gpd.GeoDataFrame, buffer_dist: List[int]
) -> gpd.GeoDataFrame:
    """
    Runs a sequence of functions to create polygons of different sizes from
    point coordinates representing different sampling sites. The coordinates
    a first projected from global EPSG 4326 to local UTM format. They are then
    buffered into polygons. Finally, the polygon coordinates are reprojected
    into the global format.

    Args:
        gdf (gpd.GeoDataFrame): Dataframe containing coordinates of sites.
        buffer_distances (list): Radii of the output polygons.

    Returns:
        gdf (gpd.GeoDataFrame): Updated dataframe with transformed coordinates
            and polygons in both local and global formats.
    """

    # Rename geometry column for clarity, since there will be multiple ones added
    gdf = gdf.rename(columns={"geometry": "global_coord"})

    # Project each Point to UTM and return coords + EPSG codes
    gdf[["utm_coord", "epsg_code"]] = gdf.apply(
        lambda row: project_to_local_utm(row["global_coord"]),
        axis=1,
        result_type="expand",
    )

    # Buffer different sized polygons and append as new columns
    for dist in buffer_dist:
        gdf[f"utm_{dist}km"] = buffer_points_in_utm(gdf["utm_coord"], dist * 1000)

    # Reproject the polygons to global coordinate format
    for dist in buffer_dist:
        gdf[f"glob_{dist}km"] = gdf.apply(
            lambda row: reproject_to_global(row[f"utm_{dist}km"], row["epsg_code"]),
            axis=1,
        )

    return gdf

In [146]:
def save_buffered_site_files(gdf, buffer_dist):
    """Saves the global polygons of each buffer radius into a separate Shapefile."""

    for dist in buffer_dist:
        gdf_res = gdf[["SSBS", f"glob_{dist}km"]].rename(
            columns={f"glob_{dist}km": "geometry"}
        )
        gdf_res.to_file(
            f"../../data/PREDICTS/site_coordinates/site_coord_buff_{buffer_dist}km.shp"
        )

### Load sampling site data and created buffered polygons

In [201]:
gdf_sites = gpd.read_file("../../data/PREDICTS/site_coordinates/site_coord.shp")
gdf_sites.head(10)

,SSBS,UN_region,geometry
0,AD1_2001__Liow 1 1,Asia,POINT (103.77861 1.35194)
1,AD1_2001__Liow 1 2,Asia,POINT (103.80806 1.35472)
2,AD1_2001__Liow 1 3,Asia,POINT (103.81167 1.39472)
3,AD1_2001__Liow 1 4,Asia,POINT (103.78722 1.32694)
4,AD1_2001__Liow 1 5,Asia,POINT (103.80361 1.28278)
5,AD1_2001__Liow 2 1,Asia,POINT (103.77861 1.35194)
6,AD1_2001__Liow 2 2,Asia,POINT (103.80806 1.35472)
7,AD1_2001__Liow 2 3,Asia,POINT (103.81167 1.39472)
8,AD1_2001__Liow 2 4,Asia,POINT (103.78722 1.32694)
9,AD1_2001__Liow 2 5,Asia,POINT (103.80361 1.28278)


In [139]:
gdf_sites.shape

(35736, 3)

In [140]:
# Set buffer distances
buffer_dist_km = [1, 10, 50]

In [202]:
start = time.time()

# Run the projection-buffering-reprojection procedure
gdf_sites = run_site_buffering(gdf_sites, buffer_dist_km)

# Saved one shapefile for each buffer radius
save_buffered_site_files(gdf_sites, buffer_dist_km)

end = time.time()
runtime = str(timedelta(seconds=end - start))
print(runtime)

0:04:32.321175


In [203]:
gdf_sites.shape

(35736, 11)

In [204]:
gdf_sites.head(10)

,SSBS,UN_region,global_coord,utm_coord,epsg_code,utm_1km,utm_10km,utm_50km,glob_1km,glob_10km,glob_50km
0,AD1_2001__Liow 1 1,Asia,POINT (103.77861 1.35194),POINT (364117.2270177193 149464.9354869793),EPSG:32648,"POLYGON ((365117.2270177193 150464.9354869793,...","POLYGON ((374117.2270177193 159464.9354869793,...","POLYGON ((414117.2270177193 199464.9354869793,...","POLYGON ((103.78759 1.36099, 103.78760 1.34290...","POLYGON ((103.86844 1.44244, 103.86852 1.26153...","POLYGON ((104.22784 1.80445, 104.22812 0.89981..."
1,AD1_2001__Liow 1 2,Asia,POINT (103.80806 1.35472),POINT (367393.6320973808 149770.4037510926),EPSG:32648,"POLYGON ((368393.6320973808 150770.4037510926,...","POLYGON ((377393.6320973808 159770.4037510926,...","POLYGON ((417393.6320973808 199770.4037510926,...","POLYGON ((103.81704 1.36377, 103.81705 1.34568...","POLYGON ((103.89789 1.44522, 103.89797 1.26431...","POLYGON ((104.25729 1.80723, 104.25757 0.90258..."
2,AD1_2001__Liow 1 3,Asia,POINT (103.81167 1.39472),POINT (367797.6346832808 154192.390200358),EPSG:32648,"POLYGON ((368797.6346832808 155192.390200358, ...","POLYGON ((377797.6346832808 164192.390200358, ...","POLYGON ((417797.6346832808 204192.390200358, ...","POLYGON ((103.82065 1.40377, 103.82066 1.38568...","POLYGON ((103.90150 1.48522, 103.90158 1.30431...","POLYGON ((104.26091 1.84723, 104.26119 0.94258..."
3,AD1_2001__Liow 1 4,Asia,POINT (103.78722 1.32694),POINT (365074.0127648303 146700.5662553131),EPSG:32648,"POLYGON ((366074.0127648303 147700.5662553131,...","POLYGON ((375074.0127648303 156700.5662553131,...","POLYGON ((415074.0127648303 196700.5662553131,...","POLYGON ((103.79620 1.33599, 103.79621 1.31790...","POLYGON ((103.87705 1.41744, 103.87713 1.23653...","POLYGON ((104.23645 1.77945, 104.23673 0.87480..."
4,AD1_2001__Liow 1 5,Asia,POINT (103.80361 1.28278),POINT (366895.2831269228 141816.8386431553),EPSG:32648,"POLYGON ((367895.2831269228 142816.8386431553,...","POLYGON ((376895.2831269228 151816.8386431553,...","POLYGON ((416895.2831269228 191816.8386431553,...","POLYGON ((103.81259 1.29183, 103.81260 1.27374...","POLYGON ((103.89344 1.37327, 103.89352 1.19236...","POLYGON ((104.25284 1.73527, 104.25310 0.83062..."
5,AD1_2001__Liow 2 1,Asia,POINT (103.77861 1.35194),POINT (364117.2270177193 149464.9354869793),EPSG:32648,"POLYGON ((365117.2270177193 150464.9354869793,...","POLYGON ((374117.2270177193 159464.9354869793,...","POLYGON ((414117.2270177193 199464.9354869793,...","POLYGON ((103.78759 1.36099, 103.78760 1.34290...","POLYGON ((103.86844 1.44244, 103.86852 1.26153...","POLYGON ((104.22784 1.80445, 104.22812 0.89981..."
6,AD1_2001__Liow 2 2,Asia,POINT (103.80806 1.35472),POINT (367393.6320973808 149770.4037510926),EPSG:32648,"POLYGON ((368393.6320973808 150770.4037510926,...","POLYGON ((377393.6320973808 159770.4037510926,...","POLYGON ((417393.6320973808 199770.4037510926,...","POLYGON ((103.81704 1.36377, 103.81705 1.34568...","POLYGON ((103.89789 1.44522, 103.89797 1.26431...","POLYGON ((104.25729 1.80723, 104.25757 0.90258..."
7,AD1_2001__Liow 2 3,Asia,POINT (103.81167 1.39472),POINT (367797.6346832808 154192.390200358),EPSG:32648,"POLYGON ((368797.6346832808 155192.390200358, ...","POLYGON ((377797.6346832808 164192.390200358, ...","POLYGON ((417797.6346832808 204192.390200358, ...","POLYGON ((103.82065 1.40377, 103.82066 1.38568...","POLYGON ((103.90150 1.48522, 103.90158 1.30431...","POLYGON ((104.26091 1.84723, 104.26119 0.94258..."
8,AD1_2001__Liow 2 4,Asia,POINT (103.78722 1.32694),POINT (365074.0127648303 146700.5662553131),EPSG:32648,"POLYGON ((366074.0127648303 147700.5662553131,...","POLYGON ((375074.0127648303 156700.5662553131,...","POLYGON ((415074.0127648303 196700.5662553131,...","POLYGON ((103.79620 1.33599, 103.79621 1.31790...","POLYGON ((103.87705 1.41744, 103.87713 1.23653...","POLYGON ((104.23645 1.77945, 104.23673 0.87480..."
9,AD1_2001__Liow 2 5,Asia,POINT (103.80361 1.28278),POINT (366895.2831269228 141816.8386431553),EPSG:32648,"POLYGON ((367895.28

### Extraction of raster data based on polygons

In [170]:
def calculate_raster_stats(
    polygon_path, raster_path, metric="mean", include_all_pixels=True
):

    # Calculate zonal statistics
    stats = zonal_stats(
        vectors=polygon_path,
        raster=raster_path,
        stats=metric,
        all_touched=include_all_pixels,
    )

    # Extract stats from each dictionary in the output list
    result = [x[metric] for x in stats]

    return result

In [228]:
def calculate_raster_stats_pgp(polygon_path, raster_path):

    # Calculate zonal statistics
    stats = pgp.geoprocessing.zonal_statistics(
        base_raster_path_band=(raster_path, 1),
        aggregate_vector_path=polygon_path,
        polygons_might_overlap=False,
    )

    # Open our vector dataset and get the layer of the vector
    vector = gdal.OpenEx(polygon_path, gdal.OF_VECTOR)
    layer = vector.GetLayer()

    # Iterate over each polygon feature in the vector and get FID
    fid_list = []
    for feature in layer:
        fid_list.append(feature.GetFID())

    stats_list = list(stats.keys())
    if fid_list == stats_list:
        print("The FID and stats dict keys are equal")
    else:
        print("The FID and stats dict keys are NOT equal")

    # Extract stats from each nested dictionary in the output
    result = []
    for fid in fid_list:
        try:
            result.append(stats[fid]["sum"] / stats[fid]["count"])
        except ZeroDivisionError:
            result.append(0)

    return result

In [232]:
def run_raster_extraction(gdf, polygon_paths, raster_paths, res_col_names):

    gdf_res = gdf[["SSBS", "UN_region", "global_coord"]].copy()

    i = 0
    for polygon_path in polygon_paths[:1]:
        for raster_path in pop_density_paths[:1]:
            start = time.time()

            result = calculate_raster_stats(
                polygon_path, raster_path, metric="mean", include_all_pixels=True
            )

            # result = calculate_raster_stats_pgp(polygon_path, raster_path)

            gdf_res.loc[:, res_col_names[i]] = result
            i += 1
            end = time.time()
            runtime = str(timedelta(seconds=end - start))
            print(runtime)

    return gdf_res

### Extract population density data from Gridded Population of the World (GPW), v4

https://sedac.ciesin.columbia.edu/data/set/gpw-v4-population-density-adjusted-to-2015-unwpp-country-totals-rev11

Unit: Number of people per square kilometer.

In [158]:
pop_density_paths = [
    "../../data/GPW/gpw_v4_2000_30_sec.tif",
    "../../data/GPW/gpw_v4_2005_30_sec.tif",
    "../../data/GPW/gpw_v4_2010_30_sec.tif",
    "../../data/GPW/gpw_v4_2015_30_sec.tif",
    "../../data/GPW/gpw_v4_2020_30_sec.tif",
]

polygon_paths = [
    "../../data/PREDICTS/site_coordinates/site_coord_buff_1km.shp",
    "../../data/PREDICTS/site_coordinates/site_coord_buff_10km.shp",
    "../../data/PREDICTS/site_coordinates/site_coord_buff_50km.shp",
]

res_col_names = [
    "_".join(combination)
    for combination in product(
        ["Pop_density"],
        ["1km", "10km", "50km"],
        ["2000", "2005", "2010", "2015", "2020"],
    )
]

In [229]:
gdf_pop_density = run_raster_extraction(
    gdf_sites, polygon_paths, pop_density_paths, res_col_names
)

The FID and stats dict keys are equal
0:00:42.200449


In [230]:
# Output from pygeoprocessing
gdf_pop_density.head(10)

,SSBS,UN_region,global_coord,Pop_density_1km_2000
0,AD1_2001__Liow 1 1,Asia,POINT (103.77861 1.35194),8665.053819
1,AD1_2001__Liow 1 2,Asia,POINT (103.80806 1.35472),1131.090088
2,AD1_2001__Liow 1 3,Asia,POINT (103.81167 1.39472),4979.729167
3,AD1_2001__Liow 1 4,Asia,POINT (103.78722 1.32694),4332.883681
4,AD1_2001__Liow 1 5,Asia,POINT (103.80361 1.28278),8465.158203
5,AD1_2001__Liow 2 1,Asia,POINT (103.77861 1.35194),9535.425456
6,AD1_2001__Liow 2 2,Asia,POINT (103.80806 1.35472),85.341309
7,AD1_2001__Liow 2 3,Asia,POINT (103.81167 1.39472),483.975861
8,AD1_2001__Liow 2 4,Asia,POINT (103.78722 1.32694),4334.962402
9,AD1_2001__Liow 2 5,Asia,POINT (103.80361 1.28278),9128.620768


In [231]:
# Output from pygeoprocessing
gdf_pop_density.tail(10)

,SSBS,UN_region,global_coord,Pop_density_1km_2000
35726,YY1_2018__Guillemot 1 61,Asia,POINT (75.46528 12.21315),590.265299
35727,YY1_2018__Guillemot 1 62,Asia,POINT (75.46528 12.21315),590.265320
35728,YY1_2018__Guillemot 1 63,Asia,POINT (75.34377 12.22568),590.265218
35729,YY1_2018__Guillemot 1 64,Asia,POINT (75.34379 12.22570),590.265289
35730,YY1_2018__Guillemot 1 65,Asia,POINT (75.48013 12.19223),590.265340
35731,YY1_2018__Guillemot 1 66,Asia,POINT (75.48022 12.19213),590.265381
35732,YY1_2018__Guillemot 1 67,Asia,POINT (75.48012 12.21315),590.265320
35733,YY1_2018__Guillemot 1 7,Asia,POINT (75.52462 12.26442),193.610229
35734,YY1_2018__Guillemot 1 8,Asia,POINT (75.52417 12.26392),193.610229
35735,YY1_2018__Guillemot 1 9,Asia,POINT (75.52428 12.26368),110.101948


In [224]:
# Output from rasterstats
gdf_pop_density.head(10)

,SSBS,UN_region,global_coord,Pop_density_1km_2000
0,AD1_2001__Liow 1 1,Asia,POINT (103.77861 1.35194),8665.053819
1,AD1_2001__Liow 1 2,Asia,POINT (103.80806 1.35472),1131.090088
2,AD1_2001__Liow 1 3,Asia,POINT (103.81167 1.39472),4979.729167
3,AD1_2001__Liow 1 4,Asia,POINT (103.78722 1.32694),4332.883681
4,AD1_2001__Liow 1 5,Asia,POINT (103.80361 1.28278),8465.158203
5,AD1_2001__Liow 2 1,Asia,POINT (103.77861 1.35194),8665.053819
6,AD1_2001__Liow 2 2,Asia,POINT (103.80806 1.35472),1131.090088
7,AD1_2001__Liow 2 3,Asia,POINT (103.81167 1.39472),4979.729167
8,AD1_2001__Liow 2 4,Asia,POINT (103.78722 1.32694),4332.883681
9,AD1_2001__Liow 2 5,Asia,POINT (103.80361 1.28278),8465.158203


In [225]:
gdf_pop_density.tail(10)

,SSBS,UN_region,global_coord,Pop_density_1km_2000
35726,YY1_2018__Guillemot 1 61,Asia,POINT (75.46528 12.21315),590.265299
35727,YY1_2018__Guillemot 1 62,Asia,POINT (75.46528 12.21315),590.265299
35728,YY1_2018__Guillemot 1 63,Asia,POINT (75.34377 12.22568),590.265218
35729,YY1_2018__Guillemot 1 64,Asia,POINT (75.34379 12.22570),590.265234
35730,YY1_2018__Guillemot 1 65,Asia,POINT (75.48013 12.19223),590.26534
35731,YY1_2018__Guillemot 1 66,Asia,POINT (75.48022 12.19213),590.26534
35732,YY1_2018__Guillemot 1 67,Asia,POINT (75.48012 12.21315),590.265299
35733,YY1_2018__Guillemot 1 7,Asia,POINT (75.52462 12.26442),193.610229
35734,YY1_2018__Guillemot 1 8,Asia,POINT (75.52417 12.26392),193.610229
35735,YY1_2018__Guillemot 1 9,Asia,POINT (75.52428 12.26368),193.610229


In [ ]:
# Save the dataframe as a file
df_pop_density = pd.DataFrame(gdf_pop_density.drop(columns="global_coord"))
df_pop_density.to_parquet("../../data/GPW/output/pop_density.parquet")

In [ ]:
gdf_pop_density.shape

### Calculation of road density inside polygons

In [ ]:
def intersect_sites_and_roads(gdf_sites, gdf_roads, buffer_dist):

    gdf_sites = gdf_sites.copy()

    # Create list of road linestrings
    road_linestrings = gdf_roads["utm_coord"]

    # Iterate through each buffer radius column in the site dataframe
    for dist in buffer_dist:
        site_road_len = []

        # Iterate through every site polygon of this size
        for site_polygon in gdf_sites[f"utm_{dist}km"]:
            total_len = 0

            # Calculate intersection between site polygon and all road linestrings
            for segment in road_linestrings:
                intersection = shapely.intersection(site_polygon, segment)
                if not intersection.is_empty:
                    total_len += intersection.length

            site_road_len.append(total_len)

        gdf_sites[f"Road_density_{dist}km"] = site_road_len

    return gdf_sites

In [ ]:
def intersect_sites_and_roads(gdf_sites, gdf_roads, buffer_dist):

    gdf_sites = gdf_sites.copy()

    # Create list of road linestrings
    road_linestrings = gdf_roads["utm_coord"]

    # Iterate through each buffer radius column in the site dataframe
    for dist in buffer_dist:
        site_road_len = []

        # Iterate through every site polygon of this size
        for site_polygon in gdf_sites[f"utm_{dist}km"]:
            total_len = 0

            # Calculate intersection between site polygon and all road linestrings
            intersection = MultiLineString(
                shapely.intersection(site_polygon, road_linestrings)
            )
            intersect_len = intersection.length
            site_road_len.append(total_len)

        gdf_sites[f"Road_density_{dist}km"] = site_road_len

    return gdf_sites

In [ ]:
def run_road_density_extraction():

    pass

### Calculate road density from Global Roads Open Access Data Set (gROADS), v1

https://sedac.ciesin.columbia.edu/data/set/groads-global-roads-open-access-v1

One data set available for each continent.

### Metadata description

- `OBJECTID`: Object ID
- `SourceID`: Source ID
- `Picture`: Picture
- `Exs`: Existence Category. Options include 1=Definite, 2=Doubtful, 0=Unspecified
- `Notes`: Notes
- `RoadID`: Road ID
- `ONme`: Official Road Name
- `RteNme`: Route Name
- `NtlClass`: National Inventory Road Class
- `FClass`: Functional Class with options 1=Highway, 2=Primary, 3=Secondary, 4=Tertiary, 5=Local/ Urban, 6=Trail, 7=Private, 0=Unspecified
- `Crgway`: Carriageways. Options include 1=Single, 2=Dual, 0=Unspecified
- `NumLanes`: Number of lanes
- `LneWidthM`: Lane Width in meters
- `RdWidthM`: Road Width in meters
- `AxleLoadMT`: Maximum Axle Loading in MT
- `TotLoadMT`: Maximum Total Loading in MT
- `SrfTpe`: Surface Type with options 1=Paved, 2=Gravel, 3=Dirt/Sand, 4=Steel, 5=Wood, 6=Grass, 0=Unspecified
- `SrfCond`: Surface Condition with options 1=Rough (<40kph), 2=Smooth (>40kph), 3=Snow/Ice, 4=Mud, 0=Unspecified
- `SrfPrep`: Surface Preparation with options 1=Natural Compaction, 2=Traffic Compaction, 3=Engineered Compaction, 4=Uncompacted, 0=Unspecified
- `IsSeasonal`: Affected by Season. Options include 1=Yes, 2=No, 0=Unspecified
- `CurntPrac`: Current Road Practicability with options 1=Non-motorized, 2=Motorbike, 3=4WD <3.5MT, 4=Light Truck <10MT, 5=Heavy Truck <20MT, 6=Truck + Trailer >20MT, 0=Unspecified
- `GdWthrPrac`: Good Weather Road Practicability with options 1=Non-motorized, 2=Motorbike, 3=4WD <3.5MT, 4=Light Truck <10MT, 5=Heavy Truck <20MT, 6=Truck + Trailer >20MT, 0=Unspecified
- `BdWthrPrac`: Bad Weather Road Practicability. Options include 1=Non-motorized, 2=Motorbike, 3=4WD <3.5MT, 4=Light Truck <10MT, 5=Heavy Truck <20MT, 6=Truck + Trailer >20MT, 0=Unspecified
- `SpeedLimit`: Speed Limit in Km/hr
- `CurntSpeed`: Current Average Speed
- `GnralSpeed`: General Average Speed
- `IsUndrCstr`: Is under Construction / Repairs. Options include 1=Yes, 2=No, 0=Unspecified
- `CstWrkETC`: Construction Work Estimated Completion Date
- `GradDeg`: Gradient in degrees
- `Sec`: Road Security Category. Options include 1=Category A (low risk), 2=Category B (low to medium risk), 3=Category C (medium to high risk), 4=Category D (high risk), 5=Category E (critical risk), 0=Unspecified
- `HasShouldr`: Has Shoulder. Options include 1=Yes, 2=No, 0=Unspecified
- `HasSidewalk`: Has Sidewalk. Options include 1=Yes, 2=No, 0=Unspecified
- `DrivSide`: Driving Side. Options include 1=Left, 2=Right, 0=Unspecified
- `IsElevated`: Is elevated / suspended above ground/water. Options include 1=Yes, 2=No, 0=Unspecified
- `HasMedian`: Has Median. Options include 1=Yes, 2=No, 0=Unspecified
- `OpStatus`: Operational Status. Options include 1=Open, 2=Restricted, 3=Closed, 4=Abandoned/Disused, 0=Unspecified
- `Shape_Length`: Length of segment
- `Length_KM`: Length of segment in kilometers



In [ ]:
road_paths = [
    "../../data/gROADS/americas/groads-v1-americas.shp",
    "../../data/gROADS/europe/groads-v1-europe.shp",
    "../../data/gROADS/africa/groads-v1-africa.shp",
    "../../data/gROADS/asia/groads-v1-asia.shp",
    "../../data/gROADS/oceania-east/groads-v1-oceania-east.shp",
    "../../data/gROADS/oceania-west/groads-v1-oceania-west.shp",
]

In [ ]:
# Inspect file format
gdf_roads = gpd.read_file(road_paths[0])
gdf_roads.head(2)

In [ ]:
# Remove columns that we don't need
gdf_roads = gdf_roads[["LENGTH_KM", "Shape_Leng", "geometry"]].rename(
    columns={"geometry": "global_coord"}
)

In [ ]:
gdf_roads = gdf_roads[:100]

In [ ]:
# Project each linestring to UTM
gdf_roads[["utm_coord", "epsg_code"]] = gdf_roads.apply(
    lambda row: project_to_utm(row["global_coord"]),
    axis=1,
    result_type="expand",
)

In [ ]:
gdf_roads.head()

In [ ]:
# Filter site polygons for Americas
gdf_sites_buff_americas = gdf_sites_buff[gdf_sites_buff["UN_region"] == "Americas"]

In [ ]:
# Calculate densities
gdf_road_density = intersect_sites_and_roads(
    gdf_sites_buff_americas, gdf_roads, buffer_dist_km
)

In [ ]:
gdf_road_density.head()

## WorldClim elevation data

https://worldclim.org/data/worldclim21.html#google_vignette

Same structure and resolution as the population density data. Can use the same approach to extract values for buffered sampling locations.

In [ ]:
elevation_path = "../../data/WorldClim/Elevation/wc2.1_30s_elev.tif"
pgp.get_raster_info(elevation_path)

## WorldClim bioclimatic data

https://worldclim.org/data/bioclim.html

In [ ]:
bioclim_path = "../../data/WorldClim/Bioclimatic/wc2.1_30s_bio_1.tif"
pgp.get_raster_info(bioclim_path)

## Unused code

In [ ]:
downscale_factor = 10  # Go from 1km to 10km

with rasterio.open("../../data/GPW/gpw_v4_2000_30_sec.tif") as dataset:

    # Read dataset and specify output shape
    # Use the average population density of the pixels being aggregated
    data = dataset.read(
        out_shape=(
            dataset.count,
            int(dataset.height / downscale_factor),
            int(dataset.width / downscale_factor),
        ),
        resampling=Resampling.average,
    )

    # Scale image transform
    transform = dataset.transform * dataset.transform.scale(
        (dataset.width / data.shape[-1]), (dataset.height / data.shape[-2])
    )

    # Define the metadata for the new downsampled raster
    out_meta = dataset.meta.copy()
    out_meta.update(
        {
            "driver": "GTiff",
            "height": data.shape[1],
            "width": data.shape[2],
            "transform": transform,
        }
    )

    # Write the downsampled raster to a new file
    with rasterio.open("../../data/GPW/gpw_v4_2000_10_km.tif", "w", **out_meta) as dest:
        dest.write(data)

In [ ]:
start = time.time()

# Load the raster file
raster_data = rasterio.open("../../data/GPW/gpw_v4_2000_10_km.tif")

# Find the value used for nodata
nodata_value = raster_data.nodatavals[0]

# Extract population density from the grids for each coordinate
pop_density = np.array(list(raster_data.sample(coord_list))).flatten()

# Set density to np.nan if it seems to be a nodata pixel
if nodata_value is not None:
    pop_density[np.isclose(pop_density, nodata_value, equal_nan=True)] = np.nan

# Append new column to the dataframe
gdf_sites_point = gdf_sites_point.copy()
gdf_sites_point["Pop_density_2000_10km"] = pop_density

end = time.time()
print(end - start)

### Method 1: Rasterio sample

Get data for the grid in which the coordinates fall. This can work for high resolution data, but might be inaccurate when downsampling the population density data and then matching pixels with the site coordinates.

In [ ]:
start = time.time()

# Load the raster file
raster_data = rasterio.open("../../data/GPW/gpw_v4_2000_30_sec.tif")

# Find the value used for nodata
nodata_value = raster_data.nodatavals[0]

# Extract population density from the grids for each coordinate
pop_density = np.array(list(raster_data.sample(coord_list))).flatten()

# Set density to np.nan if it seems to be a nodata pixel
if nodata_value is not None:
    pop_density[np.isclose(pop_density, nodata_value, equal_nan=True)] = np.nan

# Append new column to the dataframe
gdf_sites_point = gdf_sites.copy()
gdf_sites_point["Pop_density_2000_1km"] = pop_density

end = time.time()
print(end - start)

In [ ]:
def project_to_utm(geometry):
    try:
        if isinstance(geometry, Point):

            # Get the coordinate point values
            long, lat = geometry.x, geometry.y

            # Determine the UTM zone and hemisphere
            zone_number = int((long + 180) // 6) + 1
            epsg_code = (
                f"EPSG:{32700 + zone_number if lat < 0 else 32600 + zone_number}"
            )

            # Initialize Transformer object, and transform to UTM coordinates
            utm_transformer = Transformer.from_crs(
                "EPSG:4326", epsg_code, always_xy=True
            )
            utm_x, utm_y = utm_transformer.transform(long, lat)
            utm_coords = Point(utm_x, utm_y)

            return utm_coords, epsg_code

        elif isinstance(geometry, LineString):
            # Calculate EPSG code based on the first point's latitude and longitude
            first_point = geometry.coords[0]
            long, lat = first_point[0], first_point[1]
            zone_number = int((long + 180) // 6) + 1
            epsg_code = (
                f"EPSG:{32700 + zone_number if lat < 0 else 32600 + zone_number}"
            )

            # Initialize Transformer object, and transform to UTM coordinates
            utm_transformer = Transformer.from_crs(
                "EPSG:4326", epsg_code, always_xy=True
            )
            transformed_points = [
                utm_transformer.transform(point[0], point[1])
                for point in geometry.coords
            ]

            return LineString(transformed_points), epsg_code

    except ValueError as e:
        print(e)
        print(geometry)
        return None, None

In [ ]:
def project_to_utm(gdf, geom_type):

    # Make copy and specify geometry column
    gdf = gdf.copy().set_geometry("global_coord")

    if geom_type == "Point":

        # Get longitudes and latitudes
        coord = gdf.get_coordinates()
        gdf[["long", "lat"]] = coord[["x", "y"]]

        # Determine the UTM zone and hemisphere
        gdf["zone_number"] = ((gdf["long"] + 180) // 6 + 1).astype(int)
        gdf["epsg_code"] = np.where(
            gdf["lat"] < 0,
            32700 + gdf["zone_number"],
            32600 + gdf["zone_number"],
        ).astype(str)

        # Perform projection and drop intermediary columns
        projected_geoms = []
        for index, row in gdf.iterrows():
            transformer = Transformer.from_crs(
                "EPSG:4326", row["epsg_code"], always_xy=True
            )
            utm_x, utm_y = transformer.transform(
                row["global_coord"].x, row["global_coord"].y
            )
            projected_geoms.append(Point(utm_x, utm_y))

        gdf["coord_utm"] = gpd.GeoSeries(projected_geoms, index=gdf.index)
        gdf = gdf.drop(["long", "lat", "zone_number"], axis="columns")

        return gdf

    elif geom_type == "LineString":

        # Calculate EPSG code based on the first point's latitude and longitude
        gdf["first_point"] = gdf.apply(lambda row: row["global_coord"].coords[0])
        coord = gdf.get_coordinates()
        gdf[["long", "lat"]] = coord[["x", "y"]]

        # Determine the UTM zone and hemisphere
        gdf["zone_number"] = ((gdf["long"] + 180) // 6 + 1).astype(int)
        gdf["epsg_code"] = np.where(
            gdf["lat"] < 0,
            32700 + gdf["zone_number"],
            32600 + gdf["zone_number"],
        ).astype(str)

        # Perform projection and drop intermediary columns
        projected_geoms = []
        for index, row in gdf.iterrows():
            transformer = Transformer.from_crs(
                "EPSG:4326", row["epsg_code"], always_xy=True
            )
            utm_x, utm_y = transformer.transform(
                row["global_coord"].x, row["global_coord"].y
            )
            projected_geoms.append(Point(utm_x, utm_y))
        
        transformed_points = [
                utm_transformer.transform(point[0], point[1])
                for point in geometry.coords
            ]
        gdf = gdf.drop(["long", "lat", "zone_number"], axis="columns")

        return gdf